# Self-Driving Car Engineer Nanodegree


## Project: **Advanced Lane Finding**

The goal of this project is to write a software pipeline to identify the lane boundaries in a video. The steps to this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## Import Packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
%matplotlib inline

## Calibration Matrix and Distortion Coefficients

The images for camera calibration are stored in the folder called `camera_cal`.

In [ ]:
def rgb_to_bgr(img):
    r, g, b = cv2.split(img)
    return cv2.merge((b, g, r))

# chessboard dimensions
nx,ny = 9,6

# object points like (0,0,0), (1,0,0), (2,0,0) ...., (6,5,0)
objp = np.zeros((ny*nx,3), np.float32)
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

# arrays to store object points and image points from all images
objpoints = [] # 3D points in real world space
imgpoints = [] # 2D points in image plane

files = os.listdir("camera_cal/")

for file in files:
    img = cv2.imread("camera_cal/"+file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)
        img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
        plt.imshow(rgb_to_bgr(img))
        cv2.imwrite("output_camera_cal/"+file, img)

img = cv2.imread('camera_cal/calibration1.jpg')
shape = (img.shape[1], img.shape[0])
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, shape, None, None)

## Correcting for Distortion

The images to be corrected are stored in the folder called `test_images`.

In [ ]:
# mtx and dist from previous code cell

files = os.listdir("test_images/")

for file in files:
    img = cv2.imread("test_images/"+file)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    plt.imshow(rgb_to_bgr(img))
    cv2.imwrite("output_images/undist_"+file, undist)

## Color and Gradient Thresholding

In [ ]:
def apply_threshold(gray, thresh):
    binary = np.zeros_like(gray)
    binary[(gray >= thresh[0]) & (gray <= thresh[1])] = 1
    return binary

def abs_sobel_thresh(img, orient='x', thresh=(0,255)):
    thresh_min, thresh_max = thresh
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0) if orient == 'x' else cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    binary_output = apply_threshold(scaled_sobel, thresh)
    return binary_output

def color_thresh(img, sthresh=(0,255), vthresh=(0,255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_binary = apply_threshold(s_channel, sthresh)

    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    v_channel = hsv[:,:,2]
    v_binary = apply_threshold(v_channel, vthresh)

    s_and_v_binary = np.zeros_like(gray)
    s_and_v_binary[(s_binary == 1) & (v_binary == 1)] = 1
    return s_and_v_binary

def rgb_to_mono(img):
    gradx = abs_sobel_thresh(img, orient='x', thresh=(25,255))
    grady = abs_sobel_thresh(img, orient='y', thresh=(25,255))
    sv = color_thresh(img, sthresh=(100,255), vthresh=(50,255))
    mono = np.zeros_like(img[:,:,0])
    mono[(gradx == 1) & (grady == 1) | (sv == 1)] = 255
    return mono

# mtx and dist from previous code cell

for file in files:
    img = cv2.imread("test_images/"+file)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    mono = rgb_to_mono(undist)
    plt.imshow(mono, cmap="gray")
    cv2.imwrite("output_images/mono_"+file, mono)

## Perspective Transform

In [ ]:
def get_trapezoid(shape):
    w, h = shape[1], shape[0]
    p1, p4 = (w*305//1000, h*630//1000), (0, h-1)
    p3, p2 = (w*735//1000, h-1), (w*385//1000, h*630//1000)
    return [p1, p2, p3, p4]

def right_shift(quad, offset):
    return [(x+offset, y) for (x, y) in quad]

def draw_quad(quad, img):
    img = np.copy(img)
    [p1, p2, p3, p4] = quad
    color, thickness = [0, 0, 255], 3
    cv2.line(img, p1, p2, color, thickness)
    cv2.line(img, p2, p3, color, thickness)
    cv2.line(img, p3, p4, color, thickness)
    cv2.line(img, p4, p1, color, thickness)
    plt.imshow(rgb_to_bgr(img))

# mtx and dist from previous code cell

input = cv2.imread("test_images/test5.jpg")
undist = cv2.undistort(input, mtx, dist, None, mtx)
mono = rgb_to_mono(undist)
rgb = cv2.cvtColor(mono, cv2.COLOR_GRAY2RGB)
trapezoid = right_shift(get_trapezoid(rgb.shape), 200)
draw_quad(trapezoid, rgb)

In [ ]:
def get_rectangle(shape):
    w, h = shape[1], shape[0]
    p1, p4 = (0, 2), (0, h-1)
    p3, p2 = (w*735//1000, h-1), (w*735//1000, 2)
    return [p1, p2, p3, p4]

def slim(quad, offset):
    [(x1, y1), (x2, y2), (x3, y3), (x4, y4)] = quad
    return [(x1+offset, y1), (x2-offset, y2),
            (x3-offset, y3), (x4+offset, y4)]

# trapezoid and mono from previous code cell

rectangle = get_rectangle(input.shape)
rectangle = right_shift(rectangle, 200)
rectangle = slim(rectangle, 150)

src = np.float32(trapezoid)
dst = np.float32(rectangle)
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

img_size = (mono.shape[1], mono.shape[0])
warped = cv2.warpPerspective(mono, M, img_size)
rgb = cv2.cvtColor(warped, cv2.COLOR_GRAY2RGB)
draw_quad(rectangle, rgb)

In [ ]:
# mtx, dist and M from previous code cell

for file in files:
    img = cv2.imread("test_images/"+file)
    img_size = (img.shape[1], img.shape[0])
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    mono = rgb_to_mono(undist)
    warped = cv2.warpPerspective(mono, M, img_size)
    plt.imshow(warped, cmap="gray")
    cv2.imwrite("output_images/warped_"+file, warped)

## Lane Finding

In [ ]:
def get_win_bounds(window, leftx_current, rightx_current, img_height, window_height, margin):
    # Identify window boundaries in x and y (and right and left)
    win_y_low = img_height - (window+1)*window_height
    win_y_high = img_height - window*window_height
    win_xleft_low = leftx_current - margin
    win_xleft_high = leftx_current + margin
    win_xright_low = rightx_current - margin
    win_xright_high = rightx_current + margin
    return [win_y_low, win_y_high,
            win_xleft_low, win_xleft_high, win_xright_low, win_xright_high]

def draw_window(window, bounds, out_img):
    # Draw the windows on the visualization image
    [win_y_low, win_y_high,
     win_xleft_low, win_xleft_high, win_xright_low, win_xright_high] = bounds
    cv2.rectangle(out_img, (win_xleft_low,win_y_low), (win_xleft_high,win_y_high), (0,255,0), 2) 
    cv2.rectangle(out_img, (win_xright_low,win_y_low), (win_xright_high,win_y_high), (0,255,0), 2)

def append_nonzero_pixels(nonzeroy, nonzerox, bounds, left_lane_inds, right_lane_inds):
    # Identify the nonzero pixels in x and y within the window #
    [win_y_low, win_y_high,
     win_xleft_low, win_xleft_high, win_xright_low, win_xright_high] = bounds
    good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
    (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
    good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
    (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
    
    # Append these indices to the lists
    left_lane_inds.append(good_left_inds)
    right_lane_inds.append(good_right_inds)
    return good_left_inds, good_right_inds

def find_lane_pixels(binary_warped, draw_windows=False, nwindows=9, margin=100, minpix=50):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Set width and height of windows - based on nwindows above and image shape
    window_width = np.int(binary_warped.shape[1]//nwindows)
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy, nonzerox = np.array(nonzero[0]), np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current, rightx_current = leftx_base, rightx_base
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds, right_lane_inds = [], []
    
    for window in range(nwindows):
        bounds = get_win_bounds(window, leftx_current, rightx_current,
            binary_warped.shape[0], window_height, margin)
        if draw_windows:
            draw_window(window, bounds, out_img)
        good_left_inds, good_right_inds = append_nonzero_pixels(
            nonzeroy, nonzerox, bounds, left_lane_inds, right_lane_inds)
        # If you found > minpix pixels recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx, lefty = nonzerox[left_lane_inds], nonzeroy[left_lane_inds] 
    rightx, righty = nonzerox[right_lane_inds], nonzeroy[right_lane_inds]

    # Color in the left and right lane regions
    if draw_windows:
        out_img[lefty, leftx] = [255, 0, 0]
        out_img[righty, rightx] = [0, 0, 255]

    return leftx, lefty, rightx, righty, out_img

def fit_polynomial(binary_warped, draw_windows=False):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(
        binary_warped, draw_windows, nwindows=9, margin=100, minpix=50)

    # Fit a second order polynomial to each lane line
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    left_fitx = np.array(left_fitx, np.int32)
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    right_fitx = np.array(right_fitx, np.int32)

    return out_img, left_fitx, right_fitx

def draw_lane_lines(img, left_fitx, right_fitx, thickness):
    lane_lines = np.zeros_like(img)
    white_lines = np.zeros_like(img)
    img_height = img.shape[0]
    ploty = np.linspace(0, img_height-1, img_height)
    left_lane = np.array(list(zip(
        np.concatenate((left_fitx-thickness,left_fitx[::-1]+thickness), axis=0),
        np.concatenate((ploty,ploty[::-1]), axis=0))), np.int32)
    right_lane = np.array(list(zip(
        np.concatenate((right_fitx-thickness,right_fitx[::-1]+thickness), axis=0),
        np.concatenate((ploty,ploty[::-1]), axis=0))), np.int32)
    cv2.fillPoly(lane_lines, [left_lane], color=[0, 0, 255])
    cv2.fillPoly(lane_lines, [right_lane], color=[255, 0, 0])
    cv2.fillPoly(white_lines, [left_lane], color=[255, 255, 255])
    cv2.fillPoly(white_lines, [right_lane], color=[255, 255, 255])
    return lane_lines, white_lines

def draw_inner_lane(img, left_fitx, right_fitx, thickness):
    img_height = img.shape[0]
    ploty = np.linspace(0, img_height-1, img_height)
    inner_lane = np.array(list(zip(
        np.concatenate((left_fitx+thickness,right_fitx[::-1]-thickness), axis=0),
        np.concatenate((ploty,ploty[::-1]), axis=0))), np.int32)
    cv2.fillPoly(img, [inner_lane], color=[0, 255, 0])

# mtx, dist and M from previous code cells

for file in files:
    img = cv2.imread("test_images/"+file)
    img_size = (img.shape[1], img.shape[0])
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    mono = rgb_to_mono(undist)
    warped = cv2.warpPerspective(mono, M, img_size)
    out_img, left_fitx, right_fitx = fit_polynomial(warped, draw_windows=True)
    lane_lines, white_lines = draw_lane_lines(
        out_img, left_fitx, right_fitx, thickness=img_size[1]/100)
    base = cv2.addWeighted(out_img, 1.0, white_lines, -1.0, 0.0)
    combined = cv2.addWeighted(base, 1.0, lane_lines, 1.0, 0.0)
    plt.imshow(combined)
    cv2.imwrite("output_images/windows_"+file, combined)

## Curvature

In [ ]:
def meters_from_center(left_fitx, right_fitx, img_size, ym_per_pix, xm_per_pix):
    camera_center = (left_fitx[-1] + right_fitx[-1])/2
    center_diff = (camera_center-img_size[0]/2)*xm_per_pix
    side = 'left' if center_diff <= 0 else 'right'
    return center_diff, side

def measure_curvature_real(binary_warped, ym_per_pix, xm_per_pix):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit polynomials to x,y in world space
    left_fit_cr = np.polyfit(np.array(lefty, np.float32)*ym_per_pix, np.array(leftx, np.float32)*xm_per_pix, 2)
    right_fit_cr = np.polyfit(np.array(righty, np.float32)*ym_per_pix, np.array(rightx, np.float32)*xm_per_pix, 2)

    img_height = binary_warped.shape[0]
    y_eval = img_height-1*ym_per_pix
    
    left_curverad = ((1+(2*left_fit_cr[0]*y_eval+left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1+(2*right_fit_cr[0]*y_eval+right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
        
    return left_curverad, right_curverad

# mtx, dist and M from previous code cells

input = cv2.imread("test_images/test5.jpg")
undist = cv2.undistort(input, mtx, dist, None, mtx)
mono = rgb_to_mono(undist)
img_size = (mono.shape[1], mono.shape[0])
warped = cv2.warpPerspective(mono, M, img_size)

ym_per_pix, xm_per_pix = 27./720, 3.7/900
ignore, left_fitx, right_fitx = fit_polynomial(warped)
center_diff, side = meters_from_center(
    left_fitx, right_fitx, img_size, ym_per_pix, xm_per_pix)
left_curverad, right_curverad = measure_curvature_real(
    warped, ym_per_pix, xm_per_pix)

plt.imshow(warped, cmap="gray")
print("Vehicle is", center_diff, "m", side, "of center")
print("Radius of curvature =", left_curverad, "m")

## Reverse Perspective Transform

In [ ]:
# mtx, dist, M and Minv from previous code cells

def undist_warp_fit(img, img_size):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    mono = rgb_to_mono(undist)
    warped = cv2.warpPerspective(mono, M, img_size)
    ignore, left_fitx, right_fitx = fit_polynomial(warped)
    return undist, warped, left_fitx, right_fitx

def draw_lane(undist, img_size, left_fitx, right_fitx):
    lines_warped = np.zeros_like(undist)
    thickness = img_size[1]/50
    lane_lines, white_lines = draw_lane_lines(
        lines_warped, left_fitx, right_fitx, thickness)
    draw_inner_lane(lane_lines, left_fitx, right_fitx, thickness)
    lane_lines = cv2.warpPerspective(lane_lines, Minv, img_size, flags=cv2.INTER_LINEAR)
    white_lines = cv2.warpPerspective(white_lines, Minv, img_size, flags=cv2.INTER_LINEAR)
    base = cv2.addWeighted(undist, 1.0, white_lines, -1.0, 0.0)
    combined = cv2.addWeighted(base, 1.0, lane_lines, 1.0, 0.0)
    return combined

def draw_pos_curvature(warped, combined, img_size, left_fitx, right_fitx):
    ym_per_pix, xm_per_pix = 27./720, 3.7/900
    center_diff, side = meters_from_center(
        left_fitx, right_fitx, img_size, ym_per_pix, xm_per_pix)
    left_curverad, right_curverad = measure_curvature_real(
        warped, ym_per_pix, xm_per_pix)
    cv2.putText(combined, "Radius of Curvature = "+str(int(round(left_curverad, 3)))+"m", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(combined, "Vehicle is "+str(abs(round(center_diff, 3)))+"m "+side+" of center", (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
def process_image(img):
    img_size = (img.shape[1], img.shape[0])
    undist, warped, left_fitx, right_fitx = undist_warp_fit(img, img_size)
    combined = draw_lane(undist, img_size, left_fitx, right_fitx)
    draw_pos_curvature(warped, combined, img_size, left_fitx, right_fitx)
    return combined

for file in files:
    img = cv2.imread("test_images/"+file)
    out_img = process_image(img)
    plt.imshow(rgb_to_bgr(out_img))
    cv2.imwrite("output_images/"+file, out_img)

## Video Pipeline

In [ ]:
from moviepy.editor import VideoFileClip

in_clip = VideoFileClip("project_video.mp4")
out_clip = in_clip.fl_image(process_image)
out_clip.write_videofile("output_video.mp4", audio=False)